In [ ]:
# train_and_register.ipynb

import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 1. Set MLflow tracking URI
mlflow.set_tracking_uri("http://localhost:5000")  # or the Docker service name 'mlflow:5000' if running inside container

# 2. Prepare data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Let's only keep two classes for simplicity (e.g., 0 or 1).
X = X[y != 2]
y = y[y != 2]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 3. Start MLflow experiment & train model
experiment_name = "my_experiment"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="rf-run"):
    # Hyperparameters
    n_estimators = 50
    max_depth = 5

    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf.fit(X_train, y_train)

    # Evaluate
    accuracy = rf.score(X_test, y_test)

    # 4. Log params, metrics
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_metric("accuracy", accuracy)

    # 5. Log model
    mlflow.sklearn.log_model(rf, artifact_path="model", registered_model_name="my_sklearn_model")

    # The model is now in the "my_sklearn_model" name in the registry.
    # You can transition it to Production or Staging manually in MLflow UI, or programmatically:
    # from mlflow.tracking import MlflowClient
    # client = MlflowClient()
    # client.transition_model_version_stage(
    #     name="my_sklearn_model",
    #     version=1,
    #     stage="Production"
    # )

    print(f"Model logged with accuracy: {accuracy}")